In [1]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    print("Installing IDAES, GLPK, IPOPT using helper script...")
    helper.install_idaes()
    helper.install_glpk()
    helper.install_ipopt()
    print("Helper script finished.")

    # Añadir la instalación de HiGHS aquí
    print("Installing HiGHS...")
    !pip install highspy
    print("HiGHS installed.")
    print("instalando folium")
    !pip install folium
    print("folium instalado")

--2025-05-08 22:40:48--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-05-08 22:40:48 (64.2 MB/s) - ‘helper.py’ saved [7171/7171]

Installing IDAES, GLPK, IPOPT using helper script...
Installing idaes via pip...
idaes was successfully installed
Installing glpk via apt-get...
Running idaes get-extensions to install Ipopt, k_aug, and more...
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)

[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)

Couenne 0.5.8 -- an Open-Source solver for Mixe

# Caso Base

In [ ]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import random
import math
import matplotlib.pyplot as plt
import networkx as nx
import requests
import json
import folium

# --- Configuración de la API de Openrouteservice ---
ORS_API_KEY = '5b3ce3597851110001cf624895d675fa2d224e0f90bf2cb432dc61b3'
ORS_MATRIX_URL = 'https://api.openrouteservice.org/v2/matrix/driving-car'

# --- 1. Data Generation (Simulating CSV files) ---
def generate_sample_data(num_depots, num_clients, num_vehicles):
    """Generates sample dataframes for depots, clients, and vehicles."""
    depots_data = {
        'DepotID': [f'D{i+1}' for i in range(num_depots)],
        'Longitude': [random.uniform(-74.2, -73.9) for _ in range(num_depots)],
        'Latitude': [random.uniform(4.5, 4.8) for _ in range(num_depots)],
        'DepotCapacity': [random.randint(500, 1000) for _ in range(num_depots)]
    }
    depots_df = pd.DataFrame(depots_data)

    clients_data = {
        'ClientID': [f'C{i+1}' for i in range(num_clients)],
        'Demand': [random.randint(10, 50) for _ in range(num_clients)],
        'Longitude': [random.uniform(-74.2, -73.9) for _ in range(num_clients)],
        'Latitude': [random.uniform(4.5, 4.8) for _ in range(num_clients)]
    }
    clients_df = pd.DataFrame(clients_data)

    vehicles_data = {
        'VehicleID': [f'V{i+1}' for i in range(num_vehicles)],
        'Capacity': [random.randint(80, 150) for _ in range(num_vehicles)],
        'Range': [random.randint(100, 300) for _ in range(num_vehicles)] # Range in km
    }
    vehicles_df = pd.DataFrame(vehicles_data)

    depots_df.to_csv('depots.csv', index=False)
    clients_df.to_csv('clients.csv', index=False)
    vehicles_df.to_csv('vehicles.csv', index=False)
    print("Generated sample data saved to CSVs (depots.csv, clients.csv, vehicles.csv).")

    return depots_df, clients_df, vehicles_df


# --- Función para obtener matriz de distancias/tiempos de la API ---
def get_matrix_from_ors_api(locations_dict, node_ids, api_key):
    """
    Obtiene la matriz de distancias y duraciones de la API de Openrouteservice.
    locations_dict: dict {node_id: (lon, lat)}
    node_ids: lista ordenada de node_ids (Depots + Clients)
    """
    api_locations = [[locations_dict[node_id][0], locations_dict[node_id][1]] for node_id in node_ids]

    body = {"locations": api_locations, "metrics": ["distance", "duration"]}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': api_key,
        'Content-Type': 'application/json; charset=utf-8'
    }

    print(f"Calling Openrouteservice API for matrix with {len(node_ids)} locations...")
    print(f"Requesting metrics: distance, duration.")

    try:
        response = requests.post(ORS_MATRIX_URL, json=body, headers=headers, timeout=60)

        if response.status_code == 200:
            data = response.json()
            distance_matrix_meters = data.get("distances")
            duration_matrix_seconds = data.get("durations")

            if distance_matrix_meters and duration_matrix_seconds:
                print("API call successful. Matrix received.")
                distances_km = {}
                durations_sec = {}
                n = len(node_ids)
                for i in range(n):
                    for j in range(n):
                        from_node = node_ids[i]
                        to_node = node_ids[j]
                        dist_m = distance_matrix_meters[i][j] if distance_matrix_meters[i] and distance_matrix_meters[i][j] is not None else float('inf')
                        dur_s = duration_matrix_seconds[i][j] if duration_matrix_seconds[i] and duration_matrix_seconds[i][j] is not None else float('inf')

                        distances_km[(from_node, to_node)] = dist_m / 1000.0 # Convertir metros a km
                        durations_sec[(from_node, to_node)] = dur_s # Segundos

                return distances_km, durations_sec # Retornamos ambas matrices

            else:
                print("API response missing 'distances' or 'durations' data.")
                print(f"Response body: {response.text}")
                return None, None

        else:
            print(f"API call failed with status code {response.status_code}: {response.reason}")
            print(f"Response body: {response.text}")
            return None, None

    except requests.exceptions.Timeout:
        print(f"API request timed out after 60 seconds.")
        return None, None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
        return None, None
    except json.JSONDecodeError:
        print(f"API response is not valid JSON. Response body: {response.text}")
        return None, None


# --- 2. Preprocessing and Parameter Calculation (Modificada) ---
def preprocess_data(depots_df, clients_df, vehicles_df, api_key):
    """Processes raw dataframes into dictionaries for Pyomo."""
    params = {}

    locations = {}
    for _, row in depots_df.iterrows():
        locations[row['DepotID']] = (row['Longitude'], row['Latitude'])
    for _, row in clients_df.iterrows():
        locations[row['ClientID']] = (row['Longitude'], row['Latitude'])

    params['locations'] = locations

    all_nodes = depots_df['DepotID'].tolist() + clients_df['ClientID'].tolist()

    # Llama a la API para obtener las matrices de distancia y duración
    distances_km, durations_sec = get_matrix_from_ors_api(locations, all_nodes, api_key)

    if distances_km is None:
        print("Error getting distance/duration matrix from API. Cannot proceed.")
        return None

    params['distances'] = distances_km
    params['durations'] = durations_sec # Guardamos también las duraciones

    params['depot_capacity'] = dict(zip(depots_df['DepotID'], depots_df['DepotCapacity']))
    params['client_demand'] = dict(zip(clients_df['ClientID'], clients_df['Demand']))
    params['vehicle_capacity'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Capacity']))
    params['vehicle_range'] = dict(zip(vehicles_df['VehicleID'], vehicles_df['Range']))

    params['F_freight_cost'] = 5000
    params['M_maintenance_cost'] = 23
    params['Pf_fuel_cost'] = 525.1
    params['total_unit_distance_cost'] = params['F_freight_cost'] + params['M_maintenance_cost'] + params['Pf_fuel_cost']

    params['set_D'] = depots_df['DepotID'].tolist()
    params['set_C'] = clients_df['ClientID'].tolist()
    params['set_V'] = vehicles_df['VehicleID'].tolist()
    params['set_N'] = params['set_D'] + params['set_C']

    return params

# --- 3. Pyomo Model Implementation ---
def build_vrp_model(params):
    """Builds the Pyomo VRP model."""
    model = pyo.ConcreteModel(name="VehicleRoutingProblem")

    # --- SETS ---
    model.D = pyo.Set(initialize=params['set_D'])
    model.C = pyo.Set(initialize=params['set_C'])
    model.V = pyo.Set(initialize=params['set_V'])
    model.N = pyo.Set(initialize=params['set_N'])
    model.C_plus_D = model.C | model.D

    # --- PARAMETERS ---
    model.dist = pyo.Param(model.N, model.N, initialize=params['distances'])
    # model.duration = pyo.Param(model.N, model.N, initialize=params['durations']) # Descomentar si necesitas duraciones en las restricciones

    model.depot_cap = pyo.Param(model.D, initialize=params['depot_capacity'])
    model.client_demand = pyo.Param(model.C, initialize=params['client_demand'])
    model.vehicle_cap = pyo.Param(model.V, initialize=params['vehicle_capacity'])
    model.vehicle_range = pyo.Param(model.V, initialize=params['vehicle_range'])
    model.cost_per_km = pyo.Param(initialize=params['total_unit_distance_cost'])
    model.num_clients = pyo.Param(initialize=len(params['set_C']))

    # --- DECISION VARIABLES ---
    model.Y_jk = pyo.Var(model.C, model.V, domain=pyo.Binary)
    model.z_djk = pyo.Var(model.D, model.C, model.V, domain=pyo.Binary)
    model.x_c1c2k = pyo.Var(model.C, model.C, model.V, domain=pyo.Binary, rule=lambda m, c1, c2, k: m.x_c1c2k[c1,c2,k].setlb(0) if c1==c2 else None )
    model.x_prime_cdk = pyo.Var(model.C, model.D, model.V, domain=pyo.Binary)
    model.u_jk = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals)

    # --- OBJECTIVE FUNCTION (Minimize total distance cost) ---
    def objective_rule(m):
        cost = 0
        cost += sum(m.cost_per_km * m.dist[d,j] * m.z_djk[d,j,k] for d in m.D for j in m.C for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c1,c2] * m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2 for k in m.V)
        cost += sum(m.cost_per_km * m.dist[c,d] * m.x_prime_cdk[c,d,k] for c in m.C for d in m.D for k in m.V)
        return cost
    model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # --- CONSTRAINTS ---
    # 1. Client Assignment: Each client j is assigned to exactly one vehicle k.
    def client_assignment_rule(m, j):
        return sum(m.Y_jk[j,k] for k in m.V) == 1
    model.client_assignment_con = pyo.Constraint(model.C, rule=client_assignment_rule)

    # 2. Vehicle Capacity: Total demand of clients assigned to vehicle k must not exceed its capacity.
    def vehicle_capacity_rule(m, k):
        return sum(m.client_demand[j] * m.Y_jk[j,k] for j in m.C) <= m.vehicle_cap[k]
    model.vehicle_capacity_con = pyo.Constraint(model.V, rule=vehicle_capacity_rule)

    # 3a: A vehicle starts from at most one depot to at most one client.
    def vehicle_must_start_if_used_rule(m, k):
        return sum(m.z_djk[d,j,k] for d in m.D for j in m.C) <= 1
    model.vehicle_start_at_most_one_depot_con = pyo.Constraint(model.V, rule=vehicle_must_start_if_used_rule)

    # 3b: If a vehicle is used (serves any client), it must start from a depot.
    def link_Y_to_z_start_rule(m, k):
         M = m.num_clients
         return sum(m.Y_jk[j,k] for j in m.C) <= M * sum(m.z_djk[d,j,k] for d in m.D for j in m.C)
    model.link_Y_to_z_start_con = pyo.Constraint(model.V, rule=link_Y_to_z_start_rule)

    # 4. Route End: If a vehicle starts a tour, it must end at a depot.
    def vehicle_tour_end_rule(m, k):
        return sum(m.z_djk[d,j,k] for d in m.D for j in m.C) == sum(m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
    model.vehicle_tour_end_con = pyo.Constraint(model.V, rule=vehicle_tour_end_rule)

    # 5. Flow Conservation at Client Nodes for vehicle k:
    def flow_conservation_rule(m, j_node, k):
        incoming_to_j = sum(m.z_djk[d,j_node,k] for d in m.D) + sum(m.x_c1c2k[c1,j_node,k] for c1 in m.C if c1 != j_node)
        outgoing_from_j = sum(m.x_prime_cdk[j_node,d,k] for d in m.D) + sum(m.x_c1c2k[j_node,c2,k] for c2 in m.C if j_node != c2)
        return incoming_to_j == outgoing_from_j
    model.flow_conservation_con = pyo.Constraint(model.C, model.V, rule=flow_conservation_rule)

    # 6. Linking Y_jk (Assignment) to actual visit for vehicle k:
    def link_assign_to_visit_rule(m, j_node, k):
        incoming_to_j = sum(m.z_djk[d,j_node,k] for d in m.D) + sum(m.x_c1c2k[c1,j_node,k] for c1 in m.C if c1 != j_node)
        return incoming_to_j == m.Y_jk[j_node,k]
    model.link_assign_to_visit_con = pyo.Constraint(model.C, model.V, rule=link_assign_to_visit_rule)

    # 7. Depot Capacity: Total demand leaving depot d as first stop.
    def depot_capacity_rule_pdf(m, d_node):
        return sum(m.client_demand[j] * m.z_djk[d_node,j,k] for j in m.C for k in m.V) <= m.depot_cap[d_node]
    model.depot_capacity_pdf_con = pyo.Constraint(model.D, rule=depot_capacity_rule_pdf)

    # 8. Vehicle Range (Uses real distances from model.dist)
    def vehicle_range_rule(m,k):
        dist_k = 0
        dist_k += sum(m.dist[d,j] * m.z_djk[d,j,k] for d in m.D for j in m.C)
        dist_k += sum(m.dist[c1,c2] * m.x_c1c2k[c1,c2,k] for c1 in m.C for c2 in m.C if c1 != c2)
        dist_k += sum(m.dist[c,d] * m.x_prime_cdk[c,d,k] for c in m.C for d in m.D)
        return dist_k <= m.vehicle_range[k]
    model.vehicle_range_con = pyo.Constraint(model.V, rule=vehicle_range_rule)

    # 9. No self-loops for x_c1c2k (client to same client)
    def no_self_loops_clients_rule(m, c, k):
        return m.x_c1c2k[c,c,k] == 0
    model.no_self_loops_clients_con = pyo.Constraint(model.C, model.V, rule=no_self_loops_clients_rule)

    # 10. Subtour Elimination (MTZ Constraints)
    def u_bounds_lower_rule(m, j, k):
        return m.u_jk[j,k] >= 1.0 * m.Y_jk[j,k]
    model.u_bounds_lower_con = pyo.Constraint(model.C, model.V, rule=u_bounds_lower_rule)

    def u_bounds_upper_rule(m, j, k):
        return m.u_jk[j,k] <= (m.num_clients) * m.Y_jk[j,k]
    model.u_bounds_upper_con = pyo.Constraint(model.C, model.V, rule=u_bounds_upper_rule)

    def mtz_rule(m, c1, c2, k):
        if c1 == c2:
            return pyo.Constraint.Skip
        return m.u_jk[c1,k] - m.u_jk[c2,k] + (m.num_clients) * m.x_c1c2k[c1,c2,k] <= (m.num_clients) - 1.0
    model.mtz_con = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

    def mtz_initial_u_rule(m, d, j, k):
        return m.u_jk[j,k] <= 1.0 + (m.num_clients) * (1 - m.z_djk[d,j,k])
    model.mtz_initial_u_con = pyo.Constraint(model.D, model.C, model.V, rule=mtz_initial_u_rule)

    return model

# --- 4. Solving the Model and Extracting Results ---
def solve_model(model, solver_name='cbc', time_limit_seconds=None):
    """Solves the Pyomo model and returns results."""
    solver = SolverFactory(solver_name)

    if time_limit_seconds is not None:
        print(f"Setting time limit for solver '{solver_name}' to {time_limit_seconds} seconds.")
        if solver_name.lower() == 'gurobi':
            solver.options['TimeLimit'] = time_limit_seconds
        elif solver_name.lower() == 'cbc':
             solver.options['sec'] = time_limit_seconds
        elif solver_name.lower() == 'highs':
            solver.options['time_limit'] = time_limit_seconds
        else:
            print(f"Warning: Time limit option not known for solver '{solver_name}'. Time limit not set.")

    print(f"Attempting to solve with {solver_name}...")
    results = solver.solve(model, tee=True)

    return results

def extract_routes_and_details(model, results, params):
    """Extracts routes, loads, and costs from the solved model."""
    if not (results.solver.status in {pyo.SolverStatus.ok, pyo.SolverStatus.aborted}) or \
       not (results.solver.termination_condition in {pyo.TerminationCondition.optimal,
                                                     pyo.TerminationCondition.feasible,
                                                     pyo.TerminationCondition.maxTimeLimit}):
        print(f"Solver did not find an acceptable solution. Status: {results.solver.status}, Condition: {results.solver.termination_condition}")
        return None

    print("\n--- Solution Found ---")
    try:
        objective_value = pyo.value(model.total_cost)
        print(f"Total Cost: {objective_value:.2f} COP")
        solution_details = {'total_cost': objective_value, 'routes': {}}
    except Exception as e:
        print(f"Warning: Could not access objective value from model after solve: {e}.")
        print("Attempting to extract routes based on variable values...")
        solution_details = {'total_cost': float('inf'), 'routes': {}}


    for k_id in params['set_V']:
        vehicle_route = []
        vehicle_load = 0 # Calculated at the end
        vehicle_dist = 0
        current_node = None
        is_vehicle_used_for_route = False

        # Check start from depot (z_djk)
        for d_id in params['set_D']:
            for j_id in params['set_C']:
                 if pyo.value(model.z_djk[d_id,j_id,k_id]) is not None and pyo.value(model.z_djk[d_id,j_id,k_id]) > 0.5:

                    vehicle_route.append(d_id)
                    vehicle_route.append(j_id)
                    current_node = j_id
                    if (d_id, j_id) in params['distances']:
                         vehicle_dist += params['distances'][(d_id,j_id)]
                    else:
                         print(f"Warning: Distance not found for ({d_id}, {j_id}).")

                    is_vehicle_used_for_route = True
                    break
            if current_node: break

        if not is_vehicle_used_for_route:
            solution_details['routes'][k_id] = {'route': [], 'load': 0, 'distance': 0, 'clients_served': []}
            continue

        # Follow path between clients (x_c1c2k)
        clients_assigned_to_k = [j_id for j_id in params['set_C']
                                 if pyo.value(model.Y_jk[j_id,k_id]) is not None
                                 and pyo.value(model.Y_jk[j_id,k_id]) > 0.5]


        safety_counter = 0
        max_safety_steps = len(params['set_C']) * 2

        while current_node and current_node in params['set_C'] and safety_counter < max_safety_steps :

            found_next_client = False
            next_node = None

            for c2_id in params['set_C']:
                if c2_id != current_node:
                     if pyo.value(model.x_c1c2k[current_node,c2_id,k_id]) is not None and pyo.value(model.x_c1c2k[current_node,c2_id,k_id]) > 0.5:

                         if c2_id in vehicle_route:
                             print(f"Warning: Client {c2_id} would be revisited by {k_id} in current trace. Route: {vehicle_route}. Stopping trace for this vehicle.")
                             current_node = None
                             found_next_client = False
                             break

                         next_node = c2_id
                         found_next_client = True
                         break

            if found_next_client and next_node:
                 vehicle_route.append(next_node)
                 if (current_node, next_node) in params['distances']:
                     vehicle_dist += params['distances'][(current_node, next_node)]
                 else:
                     print(f"Warning: Distance not found for ({current_node}, {next_node}).")

                 current_node = next_node
                 safety_counter += 1

            else:
                break

        # Check for return to depot (x_prime_cdk) from the last client visited
        returned_to_depot = False
        last_node_in_trace = vehicle_route[-1] if vehicle_route else None

        if last_node_in_trace and last_node_in_trace in params['set_C']:
             for d_id in params['set_D']:
                 if pyo.value(model.x_prime_cdk[last_node_in_trace,d_id,k_id]) is not None and pyo.value(model.x_prime_cdk[last_node_in_trace,d_id,k_id]) > 0.5:

                    vehicle_route.append(d_id)
                    if (last_node_in_trace, d_id) in params['distances']:
                         vehicle_dist += params['distances'][(last_node_in_trace,d_id)]
                    else:
                         print(f"Warning: Distance not found for ({last_node_in_trace}, {d_id}).")

                    returned_to_depot = True
                    break

        # Final load calculation based on Y_jk (clients assigned to this vehicle)
        actual_load_for_vehicle = sum(params['client_demand'][c] for c in clients_assigned_to_k)

        # Store the extracted route details
        solution_details['routes'][k_id] = {
            'route': vehicle_route,
            'load': actual_load_for_vehicle,
            'distance': vehicle_dist,
            'clients_served': clients_assigned_to_k
        }

        # Add warnings if the extracted route seems incomplete or problematic
        if is_vehicle_used_for_route:
             clients_in_traced_route = [node for node in vehicle_route if node in params['set_C']]
             assigned_but_not_traced = [c for c in clients_assigned_to_k if c not in clients_in_traced_route]
             traced_but_not_assigned = [c for c in clients_in_traced_route if c not in clients_assigned_to_k]
             if assigned_but_not_traced:
                  print(f"Warning: Vehicle {k_id} was assigned clients {assigned_but_not_traced} but did not visit them in trace.")
             if traced_but_not_assigned:
                  print(f"Warning: Vehicle {k_id} visited clients {traced_but_not_assigned} in trace which are not assigned to it.")

             route_starts_at_depot = len(vehicle_route) > 0 and vehicle_route[0] in params['set_D']
             route_ends_at_depot = len(vehicle_route) > 0 and vehicle_route[-1] in params['set_D']

             if is_vehicle_used_for_route and not (route_starts_at_depot and route_ends_at_depot):
                 if len(vehicle_route) > 1:
                      print(f"Warning: Vehicle {k_id} was used but its traced route does not start and end at a depot. Route: {' -> '.join(vehicle_route)}")


    return solution_details

# --- 5. Visualization (NetworkX) ---
# CORREGIDA: Usar la clave correcta 'depot_capacity'
def visualize_routes(solution_details, params):
    """Visualizes the routes on a graph."""
    if not solution_details or not solution_details['routes']:
        print("No solution to visualize.")
        return

    loc_data = params['locations']
    G = nx.DiGraph()

    pos = {}
    node_colors = {}
    node_labels = {}
    node_size = {}

    for d_id in params['set_D']:
        G.add_node(d_id)
        pos[d_id] = (loc_data[d_id][0], loc_data[d_id][1])
        node_colors[d_id] = 'red'
        node_labels[d_id] = f"{d_id}\n(Cap:{params['depot_capacity'][d_id]})"
        node_size[d_id] = 3000

    for c_id in params['set_C']:
        G.add_node(c_id)
        pos[c_id] = (loc_data[c_id][0], loc_data[c_id][1])
        node_colors[c_id] = 'lightblue'
        node_labels[c_id] = f"{c_id}\n(D:{params['client_demand'][c_id]})"
        node_size[c_id] = 1500 + params['client_demand'][c_id] * 20


    plt.figure(figsize=(16, 12))
    nx.draw_networkx_nodes(G, pos,
                             node_color=[node_colors[n] for n in G.nodes()],
                             node_size=[node_size[n] for n in G.nodes()],
                             alpha=0.9, edgecolors='black', linewidths=1)

    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=7, font_weight='bold')


    edge_colors = ['b', 'g', 'c', 'm', 'y', 'k', '#ff7f0e', '#d62728']
    vehicle_styles = ['-', '--', '-.', ':']

    for node_id in params['set_D'] + params['set_C']:
        if node_id not in G.nodes():
            print(f"Warning: Node {node_id} missing from graph before drawing edges.")
            if node_id in loc_data:
                 pos[node_id] = (loc_data[node_id][0], loc_data[node_id][1])
                 G.add_node(node_id)

    print("\n--- Visualized Routes (NetworkX) ---")
    vehicle_idx = 0
    vehicles_with_routes = []

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) > 1:
            vehicles_with_routes.append(v_id)
            color = edge_colors[vehicle_idx % len(edge_colors)]
            style = vehicle_styles[vehicle_idx % len(vehicle_styles)]

            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]

                if u_node in G.nodes() and v_node in G.nodes():
                    G.add_edge(u_node, v_node, color=color, style=style, vehicle=v_id)
                else:
                    print(f"Warning: Skipping edge ({u_node} -> {v_node}) for Vehicle {v_id} as one or both nodes are not in the graph.")

            vehicle_idx += 1

    edge_list_with_attributes = []
    for u, v, data in G.edges(data=True):
        edge_list_with_attributes.append(((u, v), data.get('color', 'gray'), data.get('style', '-')))

    if edge_list_with_attributes:
         edges_to_draw = [edge[0] for edge in edge_list_with_attributes]
         colors_for_draw = [edge[1] for edge in edge_list_with_attributes]
         styles_for_draw = [edge[2] for edge in edge_list_with_attributes]

         pos_for_edges = {node: pos[node] for node in G.nodes() if node in pos}

         if pos_for_edges:
             nx.draw_networkx_edges(G, pos_for_edges,
                                    edgelist=edges_to_draw,
                                    edge_color=colors_for_draw,
                                    style=styles_for_draw,
                                    width=2.0,
                                    alpha=0.7,
                                    arrowstyle='->',
                                    arrows=True,
                                    node_size=[node_size.get(node, 1000) for node in pos_for_edges.keys()],
                                    arrowsize=20
                                    )
         else:
             print("Warning: No node positions available to draw edges.")

    handles = []
    labels = []
    vehicle_idx = 0

    for v_id in vehicles_with_routes:
         data = solution_details['routes'][v_id]
         color = edge_colors[vehicle_idx % len(edge_colors)]
         style = vehicle_styles[vehicle_idx % len(vehicle_styles)]
         handles.append(plt.Line2D([0], [0], color=color, lw=2, linestyle=style))
         labels.append(f"Veh {v_id}: Load {data['load']:.0f}/{params['vehicle_capacity'][v_id]}, Dist {data['distance']:.1f}/{params['vehicle_range'][v_id]:.1f} km")
         vehicle_idx += 1


    if handles:
        plt.legend(handles, labels, title="Vehicle Routes", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
        plt.subplots_adjust(right=0.7)


    plt.title("Vehicle Routes (from Openrouteservice API distances)")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.show()


# --- New Function for Folium Visualization ---
def visualize_routes_folium(solution_details, params):
    """Visualizes the routes on an interactive Folium map."""
    if not solution_details or not solution_details['routes']:
        print("\nNo solution to visualize with Folium.")
        return None

    print("\nGenerating Folium map...")

    loc_data = params['locations'] # {node_id: (lon, lat)}

    # Determine map center - use the first depot's location as a simple center
    # Folium expects [latitude, longitude]
    map_center = [loc_data[params['set_D'][0]][1], loc_data[params['set_D'][0]][0]] if params['set_D'] else [4.6, -74.0] # Fallback for Bogotá

    # Create base map
    m = folium.Map(location=map_center, zoom_start=12) # Adjust zoom as needed

    # Add Depot Markers
    for d_id in params['set_D']:
        lat, lon = loc_data[d_id][1], loc_data[d_id][0] # Folium expects [lat, lon]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Depot {d_id}</b><br>Capacity: {params['depot_capacity'][d_id]}",
            icon=folium.Icon(color='red', icon='home') # Use a home icon for depots
        ).add_to(m)

    # Add Client Markers
    for c_id in params['set_C']:
        lat, lon = loc_data[c_id][1], loc_data[c_id][0]
        folium.Marker(
            [lat, lon],
            popup=f"<b>Client {c_id}</b><br>Demand: {params['client_demand'][c_id]}",
            icon=folium.Icon(color='blue', icon='info-sign') # Use info-sign for clients
        ).add_to(m)

    # Add Vehicle Routes as Polylines
    route_colors = ['blue', 'green', 'purple', 'orange', 'darkred', 'cadetblue', 'darkgreen', 'darkpurple']

    vehicle_idx = 0
    for v_id, data in solution_details['routes'].items():
        route = data['route']
        if len(route) > 1:
            route_coords = []
            for node_id in route:
                 if node_id in loc_data:
                      route_coords.append([loc_data[node_id][1], loc_data[node_id][0]])
                 else:
                      print(f"Warning: Location data missing for node {node_id} in route for Vehicle {v_id}. Skipping polyline segment.")
                      pass


            if len(route_coords) > 1:
                color = route_colors[vehicle_idx % len(route_colors)]
                popup_text = f"<b>Vehicle {v_id}</b><br>"
                popup_text += f"Route: {' -> '.join(route)}<br>"
                popup_text += f"Load: {data['load']:.0f}/{params['vehicle_capacity'][v_id]}<br>"
                popup_text += f"Distance: {data['distance']:.2f} km"

                folium.PolyLine(
                    route_coords,
                    color=color,
                    weight=2.5,
                    opacity=0.8,
                    popup=popup_text
                ).add_to(m)

                vehicle_idx += 1

    print("Folium map generated.")
    return m

# --- New Function to Display Vehicle Summary ---
def display_vehicle_summary(solution_details, model_params):
    """Displays a summary table for each vehicle."""
    if not solution_details or not solution_details['routes']:
        print("\nNo solution details available to display vehicle summary.")
        return

    print("\n--- Vehicle Summary ---")

    summary_list = []

    durations_sec_matrix = model_params.get('durations')
    if durations_sec_matrix is None:
        print("Warning: Durations matrix not available in model_params. Cannot calculate total time.")

    fuel_cost_per_km = model_params.get('Pf_fuel_cost', 0)
    if 'Pf_fuel_cost' not in model_params:
         print("Warning: 'Pf_fuel_cost' not found in model_params. Fuel cost will be 0.")


    for v_id, data in solution_details['routes'].items():
        route = data['route']
        initial_load = data['load']
        clients_served = data['clients_served']
        total_distance = data['distance']

        depot_id = route[0] if route and route[0] in model_params['set_D'] else "N/A"

        route_sequence_str = " -> ".join(route) if route else "Not Used"

        demands_satisfied = initial_load

        total_time_sec = 0
        if route and durations_sec_matrix:
            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]
                if (u_node, v_node) in durations_sec_matrix:
                    total_time_sec += durations_sec_matrix[(u_node, v_node)]
                else:
                    print(f"Warning: Duration not found for segment ({u_node} -> {v_node}) for Vehicle {v_id}. Time calculation may be incomplete.")
                    pass

        calculated_fuel_cost = total_distance * fuel_cost_per_km

        summary_list.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': f"{initial_load:.0f}",
            'RouteSequence': route_sequence_str,
            'ClientsServed': ", ".join(clients_served) if clients_served else "None",
            'DemandsSatisfied': f"{demands_satisfied:.0f}",
            'TotalDistance (km)': f"{total_distance:.2f}",
            'TotalTime (s)': f"{total_time_sec:.0f}",
            'FuelCost (COP)': f"{calculated_fuel_cost:.2f}"
        })

    summary_df = pd.DataFrame(summary_list)

    if not summary_df.empty:
        print(summary_df.to_string(index=False))
    else:
        print("No vehicle summary data generated.")


# --- New Function to Save Verification File ---
def save_verification_file(solution_details, model_params, case_number):
    """Saves the vehicle summary to a verification CSV file."""
    if not solution_details or not solution_details['routes']:
        print(f"\nNo solution details available to save verification file for case {case_number}.")
        return

    print(f"\nGenerating verification file for case {case_number}...")

    verification_list = []

    durations_sec_matrix = model_params.get('durations')
    if durations_sec_matrix is None:
        print("Warning: Durations matrix not available. Total time in verification file will be 0 or inaccurate.")

    fuel_cost_per_km = model_params.get('Pf_fuel_cost', 0)

    for v_id, data in solution_details['routes'].items():
        route = data['route']
        initial_load = data['load']
        clients_served_list = data['clients_served']
        total_distance = data['distance']

        depot_id = route[0] if route and route[0] in model_params['set_D'] else "N/A"

        route_sequence_str = " -> ".join(route) if route else "Not Used"

        clients_served_count = len(clients_served_list)

        # Calculate Demands Satisfied (list of demands in order of visit)
        demands_satisfied_list = []
        if route:
            for node_id in route:
                # Only include demands for client nodes that are actually in the route
                if node_id in model_params['set_C']:
                    demands_satisfied_list.append(str(model_params['client_demand'][node_id]))
        demands_satisfied_str = "-".join(demands_satisfied_list) if demands_satisfied_list else "None"

        # Calculate Total Time from the route sequence and durations matrix
        total_time_sec = 0
        if route and durations_sec_matrix:
            for i in range(len(route) - 1):
                u_node, v_node = route[i], route[i+1]
                if (u_node, v_node) in durations_sec_matrix:
                    total_time_sec += durations_sec_matrix[(u_node, v_node)]
                else:
                    # print(f"Warning: Duration not found for segment ({u_node} -> {v_node}) for Vehicle {v_id}. Time calculation may be incomplete.")
                    pass

        # Convert total time from seconds to minutes
        total_time_min = total_time_sec / 60.0

        calculated_fuel_cost = total_distance * fuel_cost_per_km

        verification_list.append({
            'VehicleId': v_id,
            'DepotId': depot_id,
            'InitialLoad': int(initial_load), # Should be integer demand sum
            'RouteSequence': route_sequence_str,
            'ClientsServed': clients_served_count,
            'DemandsSatisfied': demands_satisfied_str,
            'TotalDistance': round(total_distance, 2), # Round to 2 decimal places
            'TotalTime': round(total_time_min, 1), # Round to 1 decimal place for minutes
            'FuelCost': round(calculated_fuel_cost, 2) # Round to 2 decimal places
        })

    verification_df = pd.DataFrame(verification_list)

    if not verification_df.empty:
        # Ensure columns are in the specified order
        csv_columns_order = [
            'VehicleId',
            'DepotId',
            'InitialLoad',
            'RouteSequence',
            'ClientsServed',
            'DemandsSatisfied',
            'TotalDistance',
            'TotalTime',
            'FuelCost'
        ]
        verification_df = verification_df[csv_columns_order]

        # Define the filename
        filename = f"verificacion_caso{case_number}.csv"

        # Save the DataFrame to CSV
        try:
            verification_df.to_csv(filename, index=False)
            print(f"Verification file saved successfully: {filename}")
        except Exception as e:
            print(f"Error saving verification file {filename}: {e}")

    else:
        print(f"No verification data generated for case {case_number}.")


# --- Main Execution ---
if __name__ == "__main__":
    print("Starting VRP Model with Openrouteservice API distances.")
    num_depots = 1
    num_clients = 24
    num_vehicles = 8
    print(f"Problem size: {num_depots} depots, {num_clients} clients, {num_vehicles} vehicles.")

    # Define the case number for the verification file
    current_case_number = 1 # You can change this for different cases

    depots_df, clients_df, vehicles_df = None, None, None
    try:
        # Check if CSVs exist, if not, sample data will be generated and saved
        depots_df = pd.read_csv('depots.csv')
        clients_df = pd.read_csv('clients.csv')
        vehicles_df = pd.read_csv('vehicles.csv')
        print("Data loaded from CSVs.")
    except FileNotFoundError:
        print("CSV files not found (depots.csv, clients.csv, vehicles.csv). Generating sample data instead.")
        depots_df, clients_df, vehicles_df = generate_sample_data(num_depots, num_clients, num_vehicles)

    if depots_df is None or clients_df is None or vehicles_df is None:
        print("Failed to load or generate data. Exiting.")
        exit()

    print("\nPreprocessing data (calling API for distances)...")
    model_params = preprocess_data(depots_df, clients_df, vehicles_df, ORS_API_KEY)

    if model_params is None:
        print("Preprocessing failed (API error?). Exiting.")
        exit()

    print("\nBuilding Pyomo model...")
    vrp_model = build_vrp_model(model_params)
    print("Model built.")

    print("\nSolving model (this might take time depending on size and solver)...")
    solver_name = 'highs'
    solve_time_limit = 900

    try:
        results = solve_model(vrp_model, solver_name=solver_name, time_limit_seconds=solve_time_limit)

        if (results.solver.status in {pyo.SolverStatus.ok, pyo.SolverStatus.aborted}) and \
           (results.solver.termination_condition in {pyo.TerminationCondition.optimal,
                                                     pyo.TerminationCondition.feasible,
                                                     pyo.TerminationCondition.maxTimeLimit}):

            print(f"\nSolver terminated with status: {results.solver.status} and condition: {results.solver.termination_condition}")

            if results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
                 print("Result: Solver reached time limit. Reporting the best feasible solution found.")
            elif results.solver.termination_condition == pyo.TerminationCondition.optimal:
                 print("Result: Optimal solution found.")
            elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
                 print("Result: Feasible solution found (optimality not proven).")

            try:
                 objective_value = pyo.value(vrp_model.total_cost)
                 print(f"\nBest feasible objective value found: {objective_value:.2f} COP")
            except Exception as e:
                 print(f"\nWarning: Could not access objective value from model after successful solve status: {e}")
                 print("Proceeding to extract routes based on variable values...")

            extracted_solution = extract_routes_and_details(vrp_model, results, model_params)

            if extracted_solution:
                # Visualize using NetworkX (your existing method) - Comentada
                #print("\nVisualizing routes (NetworkX)...")
                #visualize_routes(extracted_solution, model_params) # Ensure this function is corrected in your file

                # Visualize using Folium
                print("\nVisualizing routes (Folium)...")
                folium_map = visualize_routes_folium(extracted_solution, model_params)
                if folium_map:
                    display(folium_map)

                # Display vehicle summary table
                display_vehicle_summary(extracted_solution, model_params)

                # NEW: Save verification file
                save_verification_file(extracted_solution, model_params, case_number=current_case_number)


            else:
                print("No solution details extracted.")

        elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
            print("\nSolver terminated with condition: Infeasible.")
            print("Model is infeasible. Check constraints and data.")

        else:
            print(f"\nSolver terminated with unexpected status: {results.solver.status}, condition: {results.solver.termination_condition}")
            print("Solver did not find an acceptable solution.")
            print("Check solver output above for details.")

    except Exception as e:
        print(f"\nAn unexpected error occurred during model setup, solving, or post-processing: {e}")
        print(f"Ensure solver '{solver_name}' is installed and accessible.")
        print("Verify data loading, preprocessing, and model building steps if error occurred early.")

Starting VRP Model with Openrouteservice API distances.
Problem size: 1 depots, 24 clients, 8 vehicles.
Data loaded from CSVs.

Preprocessing data (calling API for distances)...
Calling Openrouteservice API for matrix with 25 locations...
Requesting metrics: distance, duration.
API call successful. Matrix received.

Building Pyomo model...
Model built.

Solving model (this might take time depending on size and solver)...
Setting time limit for solver 'highs' to 900 seconds.
Attempting to solve with highs...
Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 5633 rows; 5376 cols; 34944 nonzeros; 5184 integer variables (5184 binary)
Coefficient ranges:
  Matrix [1e+00, 7e+01]
  Cost   [7e+03, 4e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+03]
Presolving model
5441 rows, 5184 cols, 34752 nonzeros  0s
5402 rows, 5106 cols, 29938 nonzeros  1s

Solving MIP model with:
   5402 rows
   5106 cols (4914 binary, 0 integer, 0 implied int., 192 c


--- Vehicle Summary ---
VehicleId DepotId InitialLoad                             RouteSequence         ClientsServed DemandsSatisfied TotalDistance (km) TotalTime (s) FuelCost (COP)
       V1      D1          86                     D1 -> C4 -> C11 -> D1               C4, C11               86              85.04          8608       44655.83
       V2      D1         144        D1 -> C3 -> C18 -> C15 -> C5 -> D1      C3, C5, C15, C18              144              89.34         11378       46914.15
       V3      D1          87             D1 -> C13 -> C19 -> C16 -> D1         C13, C16, C19               87             114.28         13197       60006.09
       V4      D1          89              D1 -> C9 -> C20 -> C22 -> D1          C9, C20, C22               89              64.07          6783       33644.73
       V5      D1          75               D1 -> C2 -> C24 -> C6 -> D1           C2, C6, C24               75             128.20         13532       67320.40
       V6     N/A    